### Mansoor Nabawi

NLP final summer 2022

In [1]:
#importing necessary libraries
from glob import glob
import string
import pandas as pd
# from pandarallel import pandarallel #for parallel processing.
# pandarallel.initialize(progress_bar=True)
import numpy as np
from tqdm import tqdm
# from numba import jit
# import numba
import sys
import re
import os
import swifter #parallel computation for pandas
import xml.etree.ElementTree as ET
from lxml import html
import nltk
import pickle
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nabaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nabaw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nabaw\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

1. converting semcor files to xml in order to read them and do further processes

In [2]:
#path to the files
#path = 'c:\\Users\\nabaw\\Desktop\\NLP\\semcor1.7.1'
# add quotes around XML attributes, rename files with ".xml" extension

import sys
import re
import os

# #going through all the files and convert them to xml.
for f in glob("semcor1.7.1/semcor1.7.1/brown*/tagfiles/*"):
    c = open(f).read()
    c = re.sub(r'&', r'&amp;', c)
    for i in range(10):
        c = re.sub(r'(<[^>]+=)([^">]+)([ >])', r'\1"\2"\3', c)
    f2 = open(f + ".xml", "w")
    f2.write(c)
    f2.close()

### Reading datasets.

- in order to read the datasets we need use xml libraries and we also need to keep track of attributes of words as they are very important.
- Therefore, we create a class called word where we can save every attribute of a word in sentences.

In [2]:

class Word:
    """a class to store words attribute."""
    def __init__(self, text, pos=None, lemma=None, wnsn=None, lexsn=None, id=None):
        self.text = text
        self.pos = pos
        self.lemma = lemma
        self.wnsn = wnsn
        self.lexsn = lexsn
        self.id = id #for senseval

#### Semcor dataset.

- after converting files to xml we start reading all the files in all the folders.
- we use glob to go through all the subdirectories and read the semcor files.
- we xml library to parse the files and go through tags and read words and its attributes in each sentences.
- we create a list called documents to keep the sentences in it.

In [3]:
# let us read xml file
documents = []
for file in glob("semcor1.7.1/semcor1.7.1/brown*/tagfiles/*.xml"):
    tree = ET.parse(file)

    # get the root element
    root = tree.getroot()
    # let us read every sentence, one-by-one
    # we are ignoring the paragraph structure
    for sentence_tree in root.findall('context/p/'):
        sentence = []
    #     for every word in that sentence
        for word_tree in sentence_tree:
    #         get the word
            word = Word(word_tree.text)
            
    #         if the word xml tag contains info about pos, lemma, wnsn, lexsn, then extract it
            if 'pos' in word_tree.attrib:
                word.pos = word_tree.attrib['pos']
                
            if 'lemma' in word_tree.attrib:
                word.lemma = word_tree.attrib['lemma']
                
            if 'wnsn' in word_tree.attrib:
                word.wnsn = word_tree.attrib['wnsn']
                
            if 'lexsn' in word_tree.attrib:
                word.lexsn = word_tree.attrib['lexsn']
            
            sentence.append( word )
        documents.append(sentence)

print('Read {0} number of documents '.format(len(documents)))

Read 37176 number of documents 


we can have a look at one example

In [21]:
" ".join([x.text for x in documents[1]])

"The jury further said in term end presentments that the City_Executive_Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City_of_Atlanta '' for the manner in which the election was conducted ."

In [28]:
[l.key() for l in wn.synsets("jury")[1].lemmas()]

['jury%1:14:01::', 'panel%1:14:01::']

In [22]:
[[x.text, x.lemma, x.lexsn] for x in documents[1]]

[['The', None, None],
 ['jury', 'jury', '1:14:00::'],
 ['further', 'far', '4:02:00::'],
 ['said', 'say', '2:32:00::'],
 ['in', None, None],
 ['term', 'term', '1:28:00::'],
 ['end', 'end', '1:28:00::'],
 ['presentments', 'presentment', '1:04:00::'],
 ['that', None, None],
 ['the', None, None],
 ['City_Executive_Committee', 'group', '1:03:00::'],
 [',', None, None],
 ['which', None, None],
 ['had', 'have', '2:40:04::'],
 ['over-all', 'overall', '5:00:00:gross:00'],
 ['charge', 'charge', '1:04:03::'],
 ['of', None, None],
 ['the', None, None],
 ['election', 'election', '1:04:01::'],
 [',', None, None],
 ['``', None, None],
 ['deserves', 'deserve', '2:42:00::'],
 ['the', None, None],
 ['praise', 'praise', '1:10:00::'],
 ['and', None, None],
 ['thanks', 'thanks', '1:10:00::'],
 ['of', None, None],
 ['the', None, None],
 ['City_of_Atlanta', 'location', '1:03:00::'],
 ["''", None, None],
 ['for', None, None],
 ['the', None, None],
 ['manner', 'manner', '1:07:02::'],
 ['in', None, None],
 ['wh

we can select 5000 sentences randomly as it is asked by the paper

In [6]:
#fix seed to produce same result everytime
np.random.seed(2022)
sample_semcor = np.random.choice( documents, size=5000)

C:\Users\nabaw\AppData\Local\Temp\ipykernel_23188\3888831070.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sample_semcor = np.random.choice( documents, size=5000)


In [7]:
len(sample_semcor)

5000

In [8]:
" ".join([x.text for x in sample_semcor[10]])

"On some of the islands ' beaches the waves came_in gently ; they were steepest on the shores facing the direction of the seaquake from which the waves had come ."

#### Senseval2 dataset.

- everything the same as we read the Semcor dataset, the only difference is we are using lxml dataset as the structure of the Sensevals are a bit different.
- we keep the dataset in the doc_senseval2.

In [9]:
#senseval2
# let us read xml file
#from lxml import html
doc_senseval2 = []
for file in glob("senseval2.semcor/senseval2.semcor/wordnet1.7.1/*"):

    tree = html.fromstring(open(file).read())
    #tree = ET.parse(open(file))

    # get the root element
    #root = tree.getroot()
    # let us read every sentence, one-by-one
    for sentence_tree in tree.getchildren():
        sentence = []
        #     for every word in that sentence
        for word_tree in sentence_tree:
    #         get the word
            word = Word(word_tree.text)
            #print(word)
            
    #         if the word xml tag contains info about pos, lemma, wnsn, lexsn, then extract it
            if 'pos' in word_tree.attrib:
                word.pos = word_tree.attrib['pos']
                
            if 'lemma' in word_tree.attrib:
                word.lemma = word_tree.attrib['lemma']
                
            if 'wnsn' in word_tree.attrib:
                word.wnsn = word_tree.attrib['wnsn']
                
            if 'lexsn' in word_tree.attrib:
                word.lexsn = word_tree.attrib['lexsn']
            
            if 'id' in word_tree.attrib:
                word.id = word_tree.attrib['id']
            
            sentence.append( word )
        doc_senseval2.append(sentence)

print('Read {0} number of doc_senseval2 '.format(len(doc_senseval2)))

Read 238 number of doc_senseval2 


In [10]:
#an example

" ".join([x.text for x in doc_senseval2[10]])

'" We \'ve tried to train the youngsters , but they have their discos and their dances , and they just drift away .'

#### Senseval3 dataset.

- same as senseval2.
- the data is saved in doc_senseval3

In [11]:
#senseval3

# let us read xml file
#from lxml import html
doc_senseval3 = []
for file in glob("senseval3.semcor/senseval3.semcor/wordnet1.7.1/*"):

    tree = html.fromstring(open(file).read())
    #tree = ET.parse(open(file))

    # get the root element
    #root = tree.getroot()
    # let us read every sentence, one-by-one
    # we are ignoring the paragraph structure
    for sentence_tree in tree.getchildren():
        sentence = []
        #     for every word in that sentence
        for word_tree in sentence_tree:
    #         get the word
            word = Word(word_tree.text)
            #print(word)
            
    #         if the word xml tag contains info about pos, lemma, wnsn, lexsn, then extract it
            if 'pos' in word_tree.attrib:
                word.pos = word_tree.attrib['pos']
                
            if 'lemma' in word_tree.attrib:
                word.lemma = word_tree.attrib['lemma']
                
            if 'wnsn' in word_tree.attrib:
                word.wnsn = word_tree.attrib['wnsn']
                
            if 'lexsn' in word_tree.attrib:
                word.lexsn = word_tree.attrib['lexsn']
            
            if 'id' in word_tree.attrib:
                word.id = word_tree.attrib['id']
            
            sentence.append( word )
        doc_senseval3.append(sentence)

print('Read {0} number of doc_senseval3 '.format(len(doc_senseval3)))

Read 300 number of doc_senseval3 


In [12]:
#an example

" ".join([x.text for x in doc_senseval3[10]])

"Brakes howled and a horn blared furiously , but the man would have been hit if Phil had n't called_out to him a second before ."

#### Embeddings.

- we will read the embeddings on numpy/pandas arrasy/dataframes.
- we read the files.
- we split the data by lines "\n"
- again we split by space to create an id and a word%lexsn array.
- we keep them in a list.
- finally make a Pandas DataFrame out of them.

In [13]:
#path to data
lexemes_path = "embeddings/lexemes.txt"
mapping_path = "embeddings/mapping.txt"
sysnsets_path = "embeddings/synsets.txt"

#### Mapping

In [14]:
with open(mapping_path) as f:
    doc= f.read()

doc = doc.split("\n")
mapp_df = []
for d in doc:
    mapp_df.append(d.split())
mapp_df = pd.DataFrame(mapp_df)

In [15]:
#looking at the first 5 rows
mapp_df.head()

,0,1
0,wn-2.1-00001742-n,"entity%1:03:00::,"
1,wn-2.1-00002219-n,"thing%1:03:00::,"
2,wn-2.1-00002361-n,"anything%1:03:00::,"
3,wn-2.1-00002471-n,"something%1:03:00::,"
4,wn-2.1-00002579-n,"nothing%1:03:00::,nonentity%1:03:00::,"


#### Lexemes/Synsets

Using KeyedVectors from gensim library

In [16]:
from gensim.models.keyedvectors import KeyedVectors
#loading embeddings
lexemes_emb = KeyedVectors.load_word2vec_format(lexemes_path, binary=False)
synsets_emb = KeyedVectors.load_word2vec_format(sysnsets_path, binary=False)

### Most common Sense and Lesk Algorithm.

In [877]:
from gensim import utils
#get accuracy
def get_acc(res):
    """a function to show accuracy by finding the intersection between two sets."""
    all = 0
    corr = 0
    for r in res:
        for word in res[r]:
            #print(word)
            lemmas = set([l.key().lower() for l in word[2].lemmas()])
            lem = set(word[1].lower().split(","))
            if "none" not in word[1].split("%"):
                all += 1
            #if there are common elements between true lemmas and all lemmas in the sysnet
                if len(lem.intersection(lemmas))>0:
                    corr += 1
    return (corr/all)*100, corr, all
#making lemma
def lemma_mixer(lem, lex):
    """a function to create lemma keys out of lemma and lexsn"""
    lem = str(lem).lower()
    lex = str(lex).lower().split(";")
    for i in range(len(lex)):
        lex[i]= lem+"%"+lex[i]
    return ",".join(lex)

#a function to find the most common sense for each word
def find_most_common_sense(docs):
    #to store the data
    full_sent = {}
    #going through every sentence
    for doc in docs:
        #storing pos, text and lemmakey for each word in a list
        pos_text = [[x.pos, x.text, lemma_mixer(x.lemma, x.lexsn)] for x in doc]
        #key for our dictionary
        kk = " ".join([x.text for x in doc])
        #an empty list for words
        full_sent[kk] = []
        #going through every word and its attributes.
        for pt in pos_text:
            pos = pt[0]
            #clean the text
            text = utils.simple_preprocess(pt[1])
            #if the pos exists search for the most common sense with that pos
            if pos !=None:
                try:
                #getting the very first
                    pos = pos[0].lower()
                    syns = wn.synsets(" ".join(text), pos)[0]
                except:
                    continue
                    #syns = None
            #if there is no pos get the very first synset
            else:
                try:
                    syns = wn.synsets(text)[0]
                except:
                    continue
                    #syns = None
            #save the result in that sentece with this order: text, true lemma, synset
            full_sent[kk].append([text,pt[2], syns])
    
    return full_sent



In [741]:
ms2 = find_most_common_sense(doc_senseval2)
get_acc(ms2)

(59.86358866736621, 1141, 1906)

In [742]:
ms2[list(ms2)[0]]

[[['art'], 'art%1:09:00::', Synset('art.n.01')],
 [['is'], 'be%2:42:03::', Synset('be.v.01')],
 [['english'], 'english%1:18:00::', Synset('english.n.01')],
 [['peculiarities'],
  'peculiarity%1:09:00::,peculiarity%1:07:02::',
  Synset('peculiarity.n.01')],
 [['rest'], 'rest%1:24:00::', Synset('remainder.n.01')],
 [['world'], 'world%1:14:02::', Synset('universe.n.01')]]

In [743]:
ms3 = find_most_common_sense(doc_senseval3)
get_acc(ms3)

(59.813084112149525, 960, 1605)

In [744]:
msc = find_most_common_sense(sample_semcor)
get_acc(msc)

(66.76295593688023, 15485, 23194)

In [745]:
msc[list(msc)[0]]

[[['houghton'], 'none%none', Synset('houghton.n.01')],
 [['said'], 'say%2:32:00::', Synset('state.v.01')],
 [['had'], 'none%none', Synset('have.v.01')],
 [['been'], 'none%none', Synset('be.v.01')],
 [['set'], 'set%2:32:00::', Synset('put.v.01')],
 [['canvass'], 'none%none', Synset('canvas.n.03')],
 [['homes'], 'none%none', Synset('home.n.01')],
 [['subdivision'], 'none%none', Synset('subdivision.n.01')],
 [['is'], 'none%none', Synset('be.v.01')],
 [['located'], 'locate%2:42:00::', Synset('locate.v.01')],
 [['northeast'], 'none%none', Synset('northeast.n.01')],
 [['mile'], 'none%none', Synset('mile.n.01')],
 [['road'], 'none%none', Synset('road.n.01')]]

- we go through every sentence.
- we collect word text for every sentence.
- we exculde digits, stop_words and punctuations.
- for every word in the sentences:
    - we find the synsets using wordnet.
    - for every synset:
        - gather definition.
        - gather examples.
        - we keep both in two different dictionaries the word is the key and the values are sense and its defintion/examples.
    - we go through every word in the definition dictionary.
        -  and find the overlaps between the context and definition as well as examples.
    - the sense with the most overlps/most common words is chosen.
    - if the overlap is 0, then choose the very first one.
- keep all the data in one dictionary.
    - dictionary key is the full sentence.
    - values are the words with their senses.

In [835]:
def evaluate_accuracy(sample):
    """a function get accuracy. by finding the intersection between two sets"""
    all = 0
    corr = 0
    for s in sample:
        for word in sample[s]:
            true_ = word[1]
            all_lemmas = word[3]
            if "none" not in true_.split("%"):
                all += 1
                if len(set(true_.split(",")).intersection(set([x.lower() for x in all_lemmas])))>0:
                    corr += 1

    return (corr/all)*100, corr, all


def lesk_mc(sample_sentences):
    """Plain lesk algorithm"""
    puncs = set(string.punctuation)
    stop_words = set(stopwords.words('english'))

    from tqdm import tqdm
    full_sent_dic = {}

    #going through every sentence.
    for sample in tqdm(sample_sentences):
        #full sentence
        sentence = [[s.text, lemma_mixer(s.lemma, s.lexsn)] for s in sample] 
        #making a lemma key for each word
        #filtering sentence, no stopwords included and no "," , "."
        filtered_sentence = [word for word in sentence if word[0] not in stop_words]
        #not considering numbers 
        filtered_sentence = [x for x in filtered_sentence if not x[0].isdigit()]
        #removing punctuations
        filtered_sentence = [x for x in filtered_sentence if x[0] not in puncs]
        #a dictionary to keep definitions and examples of each words.
        sens_def_dic = {}
        # going trough each word in the filtered sentence
        for w in filtered_sentence:
            word_ = w[0]
            true_lemma = w[1]
            #find the senses for a specific w (word)
            senses = wn.synsets(word_)
            #making a list for each word to contain the definitions and examples.
            sens_def_dic[word_] = []
            for sens in senses:
                # appending all the senses and examples and true lemmas in the dictionaries.
                sens_def_dic[word_].append([sens, sens.definition(), sens.examples(), true_lemma])

        #going through each words definitions
        kkey = " ".join([s[0] for s in sentence])
        full_sent_dic[kkey] = []
        #just keeping the text
        context = [s[0] for s in filtered_sentence]
        for w in sens_def_dic:
            #print(filtered_sentence[0])
            #print(w)
            max = 0
            index = 0
            #every definition of each word
            for df in sens_def_dic[w]:
                #keeping the counts of overlaps
                cnt = 0
                #-------------------
                #overlaps between the sentence and the definion
                def_ = [w.lower() for w in df[1] if w not in stop_words]
                examples_ = " ".join(df[2]).split()
                examples_ = [exm.lower() for exm in examples_ if exm not in stop_words]
                #---------------------
                common = set(examples_).intersection(def_)
                common = set(context).intersection(common)

                #count of overlaps
                cnt += len(common)
                #if there is overlap and it is bigger than the max then this is selected as our sens
                if max < cnt:
                    max = cnt
                    index = df
                else:
                    #if no overlapping found, choose the very first definition.
                    index = sens_def_dic[w][0]
            #print(w + " : " + str(index))
            #excluding words with no meaning
            if index != 0:
                full_sent_dic[kkey].append([w,index[-1],index[0], [l.key() for l in index[0].lemmas()]])
            #i += 1
    return full_sent_dic


##### Lesk for Semcor dataset

In [836]:
semcor = lesk_mc(sample_semcor)
evaluate_accuracy(semcor)

100%|██████████| 5000/5000 [00:11<00:00, 446.14it/s]


(53.411108293177776, 12636, 23658)

In [837]:
semcor[list(semcor.keys())[0]][1]

['said',
 'say%2:32:00::',
 Synset('state.v.01'),
 ['state%2:32:00::', 'say%2:32:00::', 'tell%2:32:04::']]

Evaluation

##### Lesk for Sensevals

In [838]:
senseval2 = lesk_mc(doc_senseval2)
evaluate_accuracy(senseval2)

100%|██████████| 238/238 [00:00<00:00, 362.04it/s]


(48.87459807073955, 1064, 2177)

In [839]:
senseval3_lesk = lesk_mc(doc_senseval3)
evaluate_accuracy(senseval3_lesk)

100%|██████████| 300/300 [00:00<00:00, 448.01it/s]


(49.11072862880092, 856, 1743)

### Oele and van Noord (2017) method using the pre-trained word embeddings.

- We have embeddings for lexemes and sysnsets.
- we need to compute the embeddings for the sentences we have.
- therefore we use Word2vec to produce the embeddings.

In [22]:
#dropping NaN values
mapp_df= mapp_df.dropna().reset_index(drop=True)

In [23]:
from gensim.models import KeyedVectors
from gensim import utils
#loading embeddings
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [480]:
mapp_df.columns = ["wkey", "wordlemma"]

In [493]:
# mapp_df["keyz"] = mapp_df.apply(lambda x: lex_creator(x.wkey, x.wordlemma), axis=1)

In [866]:
from gensim import utils
from operator import itemgetter
from gensim import matutils

def get_acc(true_senses, result):
    """finding accuracy."""
    all_lems = []
    for s in true_senses:
        for l in s.lemmas():
            all_lems.append(l.key())
    try:
        res = result[:-1][0]
        f = []
        for r in res:
            for k in r.split(","):
                if len(k)>0:
                    f.append(k)
        #res = set([res.lower() for res in res.split(",") if len(res)>0])
        res = set(f)
        all_lems = set(s.lower() for s in all_lems)
        if len(all_lems.intersection(res))>0:
            return 1
        else:
            return 0
    except:
        return None

def show_acc(df):
    """a function to show final accuracy."""
    res = df.apply(lambda x: get_acc(x.true_synset, x.result), axis=1).dropna()
    corr = res.sum()
    all = res.shape[0]

    return (corr/all)*100, corr, all
    
def similarity_cosine(vec1, vec2):
    """a function to compute cosine similarity between two vectors."""
    cosine_similarity = np.dot(matutils.unitvec(vec1.reshape(1,-1)), matutils.unitvec(vec2))
    return cosine_similarity

def lemma_mixer(lem, lex):
    """a function to create lemma keys out of lemma and lexsn."""
    lem = str(lem).lower()
    lex = str(lex).lower().split(";")
    for i in range(len(lex)):
        lex[i]= lem+"%"+lex[i]
    return ",".join(lex)

def gloss_vec(x, model):
    """a function to create gloss vector for a given word using a word2vec model."""
    defs_ = []
    for syn in x:
        sc = utils.simple_preprocess(syn.definition())
        sc = [token for token in sc if token in model.index_to_key and token not in stopwords.words('english')]
        try:
            sc = np.mean(model[sc], axis=0)
        except:
            sc = np.zeros((1,300), dtype=float)
        defs_.append(sc)
    return defs_
    
def example_vec(x, model):
    """a function to create example vector for a give word using word2vec model."""
    defs_ = []
    for syn in x:
        sc = utils.simple_preprocess(".".join(syn.examples()))
        sc = [token for token in sc if token in model.index_to_key and token not in stopwords.words('english')]
        try:
            sc = np.mean(model[sc], axis=0)
        except:
            sc = np.zeros((1,300), dtype=float)
        defs_.append(sc)
    return defs_

#belows are some helper functions.
def lemma_finder(wn_syn, lem_n):
    lem_n = lem_n[0].split("%")[0]+"%"
    lem_syns = []
    for lem in wn_syn:
        lems = []
        for lem in lem.lemmas():
            if str(lem.key()).startswith(lem_n):
                lems.append(lem.key())
        if len(lems)>0:
            lem_syns.append(lems)
    return lem_syns


from operator import itemgetter
def finder_l(x, w):
    xx = set(wx for wx in x.lower().split(",") if len(wx)>0)
    w = [y[0] for y in w]
    ww = set(x.lower() for x in w if len(x)>0)
    #looking for lexemes with lemma key.
    res = bool(xx & ww)
    if res:
        return True
    else:
        return False

def mapp_finder(df, w):
    return df[df["wordlemma"].apply(lambda x: finder_l(x, w))]

def lex_creator(key, lemkey):
    keyz = []
    lemkey = [y for y in lemkey.split(",") if len(y)>0]
    for x in lemkey:
        if len(x)>0:
            keyz.append(x.split("%")[0]+"-"+key)
    return keyz


def disambiguator(word, true_lemmas, lemmas, gloss, examples, model, alpha, beta, lambda_):
    """the main function to perform disamiguation process."""
    if word not in context:
        return None
    #if no disambiguated words exist
    if len(dis_vec)==0:
        contxt_vec = model[context]
    #if we had disamibuated words earlier 
    else:                      
        alread_dis = np.concatenate(dis_vec, axis=0)
        contxt_vec = np.concatenate((alread_dis,model[context]), axis=0)

    context_mean_vec = np.mean(contxt_vec, axis=0)
    #counter to catch the first word
    i = 0
    #highest cosine similarity
    cos_high = 0
    #best syns id
    syns_id = 0
    #syns of the chosen word
    syns_word_selected = 0
    #synset wordnet
    synset_wd = 0
    s = 0
    #lem_n = true_lemmas[0].split("%")[0]
    keyz = mapp_finder(mapp_df, lemmas)
    # keyz["lex_keyz"] = keyz.apply(lambda x: lex_creator(x.wkey, x.wordlemma), axis=1)
    #lemma key ---------------
    for lem_syn in lemmas:
        gloss_vec = gloss[s]
        examples_vec = examples[s]
        if (len(gloss_vec)>0 and alpha != 0):
            score_sw = similarity_cosine(gloss_vec.flatten(), context_mean_vec.flatten()) #syns_embedding
        else:
            score_sw = 0
        # keyz = mapp_finder(mapp_df, lem_syn)
        if (len(examples_vec)>0 and lambda_ != 0):
            score_example_s = similarity_cosine(examples_vec.flatten(), context_mean_vec.flatten()) 
        else:
            score_example_s = 0

        for syn in lem_syn:
            #----------------
            #exact map index
            
            map_idx = keyz[keyz.iloc[:,1].apply(lambda x: True if syn in x.lower().split(",") else False)]
            #finding synset embedding
            try:
                map_lemmakey = map_idx.iloc[:,0].tolist()[0]
            except:
                continue
            syns_embedding = synsets_emb[map_lemmakey]
            #--------------------
            if beta != 0:
                #finding lexeme------------
                lem_n = map_idx.iloc[:,1].tolist()[0].split("%")[0]
                lex_key = lem_n+"-"+map_lemmakey
                try:
                    lexeme_embedding = lexemes_emb[lex_key]
                    score_lsw = similarity_cosine(lexeme_embedding.flatten(), context_mean_vec.flatten())
                except:
                    #continue
                    score_lsw = 0#similarity_cosine(np.zeros((1,300), dtype=float).flatten(), context_mean_vec.flatten())
            else:
                score_lsw = 0
            #---------------------------------

            #final score
            score = alpha*score_sw + beta*score_lsw + lambda_*score_example_s
            if i == 0:
                cos_high = score
                syns_id = lem_syn#map_idx.iloc[:,1].tolist()[0]
                syns_word_selected = syns_embedding#syns_embedding
                #synset_wd = wn_syn
                i += 1
            if score>cos_high:
                cos_high = score
                syns_id = lem_syn#map_idx.iloc[:,1].tolist()[0]
                syns_word_selected = syns_embedding#syns_embedding    
                #synset_wd = wn_syn       
        s += 1     

    #if i was able to disambiguate the word add it so next time only the synset embedding is used
    if i>0:
        context.remove(word)
        dis_vec.append(np.array(syns_word_selected, dtype=float).reshape(1,-1))
    return [syns_id, cos_high]

def final_wsd_(docs, model=word2vec, alpha=0.5, beta=0.1, lambda_=0.9):
    """Distirbutional Lesk algorithm, main function"""
    full = []
    for doc in tqdm(docs):
    #one sentence
        one_sent = utils.simple_preprocess(" ".join([x.text for x in doc]))
    #a list to store word, lemmakey, number of senses, sense key, syns
        _sent = []
    #every word in the sentence
        for wrd in one_sent:
        #make lemma key
            lemk = [lemma_mixer(x.lemma, x.lexsn) for x in doc if (("".join(utils.simple_preprocess(x.text)) == wrd)and((x.lexsn != None) and (x.lemma != None)))]
        # if there is lexsn and lemma
        #if len(lemk)>0:
            try:
            #making the lemma key, true one given by sentence.
                lemk = ",".join(lemk).split(",")
                syns_t = [wn.lemma_from_key(l).synset() for l in lemk]
            except:
                continue
            #lemk = None
            #using wordnet, find all the synsets for the word
            syns = wn.synsets(wrd)

            _sent.append([wrd, lemk,syns_t, len(syns),  syns])
        if len(_sent)==0:
            continue
        global dis_vec
        dis_vec = []
        #sorting
        _sent = sorted(_sent, key=itemgetter(3))
        global context
        context = [token[0] for token in _sent if token[0] in model.index_to_key and token[0] not in stopwords.words('english')]
        _sent = pd.DataFrame(_sent, columns=["word", "true_lemmas","true_synset", "frequency", "all_synsets"])
        _sent["gloss"] = _sent["all_synsets"].apply(lambda x: gloss_vec(x, model))
        _sent["examples"] = _sent["all_synsets"].apply(lambda x: example_vec(x, model))
        _sent["all_lemmas"]=_sent.apply(lambda x: lemma_finder(x.all_synsets, x.true_lemmas), axis=1)
        _sent["result"]=_sent.apply(lambda x: disambiguator(x.word, x.true_lemmas, x.all_lemmas, x.gloss, x.examples, model, alpha, beta, lambda_), axis=1)
        full.append(_sent)
    full = pd.concat(full)

    return full.dropna().reset_index(drop=True)

In [594]:
#distributional lesk on senseval3
a = final_wsd_(doc_senseval3, alpha=1, beta=1, lambda_=0)

100%|██████████| 300/300 [41:11<00:00,  8.24s/it] 


In [600]:
#distributional lesk on senseval2
b = final_wsd_(doc_senseval2, alpha=1, beta=1, lambda_=0)

100%|██████████| 238/238 [43:50<00:00, 11.05s/it] 


In [ ]:
#distributional lesk on sample semcor
c = final_wsd_(sample_semcor)

In [700]:
d = final_wsd_(doc_senseval2, alpha=0.5, beta=0.1, lambda_=1.2)

100%|██████████| 238/238 [52:08<00:00, 13.14s/it] 


In [701]:
show_acc(d)

(48.59398125308337, 985.0, 2027)

In [702]:
d1 = final_wsd_(doc_senseval2, alpha=0.8, beta=0, lambda_=0.9)

100%|██████████| 238/238 [51:48<00:00, 13.06s/it] 


In [703]:
show_acc(d1)

(47.557967439565864, 964.0, 2027)

In [704]:
d2 = final_wsd_(doc_senseval2, alpha=0, beta=0.8, lambda_=0.5)

100%|██████████| 238/238 [49:51<00:00, 12.57s/it] 


In [705]:
show_acc(d2)

(45.831277750370006, 929.0, 2027)

In [706]:
e1 = final_wsd_(doc_senseval3, alpha=0.5, beta=0.1, lambda_=1.2)

100%|██████████| 300/300 [45:38<00:00,  9.13s/it] 


In [707]:
show_acc(e1)

(41.40672782874618, 677.0, 1635)

In [708]:
e2 = final_wsd_(doc_senseval3, alpha=0.8, beta=0, lambda_=0.9)

100%|██████████| 300/300 [43:51<00:00,  8.77s/it] 


In [709]:
show_acc(e2)

(42.813455657492355, 700.0, 1635)

In [710]:
e3 = final_wsd_(doc_senseval3, alpha=0, beta=0.8, lambda_=0.5)

100%|██████████| 300/300 [44:12<00:00,  8.84s/it] 


In [711]:
show_acc(e3)

(36.391437308868504, 595.0, 1635)

In [712]:
c1 = final_wsd_(sample_semcor, alpha=0.5, beta=0.1, lambda_=1.2)

100%|██████████| 5000/5000 [12:08:28<00:00,  8.74s/it]  


In [713]:
show_acc(c1)

(40.52536852931093, 9457.0, 23336)

In [843]:
c2 = final_wsd_(sample_semcor, alpha=0.8, beta=0, lambda_=0.9)

100%|██████████| 5000/5000 [10:40:49<00:00,  7.69s/it]  


In [847]:
c2.head()

,word,true_lemmas,true_synset,frequency,all_synsets,gloss,examples,all_lemmas,result
0,located,[locate%2:42:00::],[Synset('situate.v.01')],5,"[Synset('locate.v.01'), Synset('situate.v.01')...","[[-0.072631836, 0.030517578, -0.040348597, 0.0...","[[0.02551651, 0.0703125, -0.01953125, 0.083292...","[[locate%2:40:00::], [locate%2:42:00::], [loca...","[[locate%2:42:00::], [0.6718176]]"
1,said,[say%2:32:00::],[Synset('state.v.01')],12,"[Synset('state.v.01'), Synset('allege.v.01'), ...","[[0.10583496, -0.028564453, 0.11706543, 0.1347...","[[0.024027506, -0.02319336, 0.025390625, 0.027...","[[say%2:32:00::], [say%2:32:01::], [say%2:32:0...","[[say%2:32:04::], [0.7423840704450704]]"
2,set,[set%2:32:00::],[Synset('specify.v.02')],45,"[Synset('set.n.01'), Synset('set.n.02'), Synse...","[[0.06587728, 0.04269409, 0.084360756, 0.18495...","[[0.026018415, 0.17170061, 0.113442555, 0.0655...","[[set%1:14:00::], [set%1:14:02::], [set%1:04:0...","[[set%5:00:00:arranged:00], [0.989714100853444]]"
3,framework,[framework%1:09:00::],[Synset('model.n.01')],3,"[Synset('model.n.01'), Synset('framework.n.02'...","[[0.07076416, -0.10839844, 0.08298111, 0.05541...","[[0.040754046, -0.007585798, 0.050432477, 0.09...","[[framework%1:09:00::], [framework%1:07:00::],...","[[framework%1:07:00::], [0.89532065]]"
4,apprehend,[apprehend%2:31:00::],[Synset('grok.v.01')],3,"[Synset('grok.v.01'), Synset('collar.v.01'), S...","[[0.10872396, -0.075927734, 0.015950521, 0.103...","[[0.032552082, -0.050130207, 0.06640625, -0.08...","[[apprehend%2:31:00::], [apprehend%2:35:00::],...","[[apprehend%2:31:00::], [0.8523975690929537]]"


In [851]:
show_acc(c2)

(40.13969832019198, 9367.0, 23336)

In [652]:
d[["true_lemmas","result"]]

,true_lemmas,result
0,[say%2:32:01::],"[[say%2:32:04::], 1.2213865055236965]"
1,[man%1:18:00::],"[[man%1:18:05::], 1.8875896333869828]"
2,[peer%2:39:00::],"[[peer%2:39:00::], 1.3647707283496857]"
3,[bleary%5:00:00:indistinct:00],"[[bleary%5:00:00:indistinct:00], 0.91221615797..."
4,[companion%1:18:00::],"[[companion%1:18:00::], 1.0065360311805387]"
5,[eye%1:08:00::],"[[eye%1:08:00::], 1.2380499206787665]"
6,[surprise%1:12:00::],"[[surprise%2:31:00::], 1.3377663999795915]"
7,[much%3:00:00::],"[[much%4:02:04::], 1.283877014350709]"
8,[ready%5:00:01:available:00],"[[ready%3:00:00::], 1.5897204950626866]"
9,[answer%1:04:00::],"[[answer%2:31:00::], 1.6709365958543396]"


In [595]:
show_acc(a)

(34.3520782396088, 562.0, 1636)

In [601]:
show_acc(b)

(44.466403162055336, 900.0, 2024)

In [763]:
def get_acc(true_senses, result):
    all_lems = []
    for s in true_senses:
        for l in s.lemmas():
            all_lems.append(l.key())
    try:
        res = result[:-1]
        f = []
        for r in res:
            for k in r.split(","):
                if len(k)>0:
                    f.append(k)
        #res = set([res.lower() for res in res.split(",") if len(res)>0])
        res = set(f)
        all_lems = set(s.lower() for s in all_lems)
        if len(all_lems.intersection(res))>0:
            return 1
        else:
            return 0
    except:
        return None

def show_acc(df):
    res = df.apply(lambda x: get_acc(x.true_synset, x.result), axis=1).dropna()
    corr = res.sum()
    all = res.shape[0]

    return (corr/all)*100, corr, all

In [445]:
s2_df = final_wsd_(doc_senseval2)

100%|██████████| 238/238 [45:08<00:00, 11.38s/it] 


In [446]:
show_acc(s2_df)

(44.565217391304344, 902.0, 2024)

In [462]:
s3_df = final_wsd_(doc_senseval3)

100%|██████████| 300/300 [38:29<00:00,  7.70s/it] 


In [463]:
show_acc(s3_df)

(33.924205378973106, 555.0, 1636)

In [464]:
sc_df = final_wsd_(sample_semcor)

100%|██████████| 5000/5000 [8:57:48<00:00,  6.45s/it]   


In [758]:
sc_df[["true_lemmas", "true_synset", "result"]]

,true_lemmas,true_synset,result
0,[locate%2:42:00::],[Synset('situate.v.01')],"[locate%2:40:01::,place%2:40:00::,site%2:40:00..."
1,[say%2:32:00::],[Synset('state.v.01')],"[say%2:32:13::,, 0.6240999814727821]"
2,[set%2:32:00::],[Synset('specify.v.02')],"[laid%5:00:00:arranged:00,set%5:00:00:arranged..."
3,[framework%1:09:00::],[Synset('model.n.01')],"[model%1:09:00::,framework%1:09:00::,, 1.0085602]"
4,[apprehend%2:31:00::],[Synset('grok.v.01')],"[apprehend%2:37:00::,, 0.8178087085639515]"
...,...,...,...
23938,[backer%1:18:00::],[Synset('angel.n.03')],"[angel%1:18:02::,backer%1:18:00::,, 0.45259362]"
23939,[person%1:03:00::],[Synset('person.n.01')],"[Hudson%1:18:00::,, 0.4389045449106892]"
23940,[two%1:23:00::],[Synset('two.n.01')],"[two%1:23:00::,2%1:23:00::,II%1:23:00::,deuce%..."
23941,[cape%1:17:00::],[Synset('cape.n.01')],"[cape%1:06:00::,mantle%1:06:00::,, 0.941523092..."


In [764]:
show_acc(sc_df)

(35.123354915153826, 8300.0, 23631)

### Extension, Training a Word2vec model using few samples of each dataset

In [854]:
#fix seed to produce same result everytime
np.random.seed(2022)
#choosing 50 samples of each dataset
f_semcor = np.random.choice( documents, size=50)
f_senseval2 = np.random.choice( doc_senseval2, size=50)
f_senseval3 = np.random.choice( doc_senseval3, size=50)
#------------------
#making one big dataset
altered_dataset = []
for s, s2, s3 in zip(f_semcor, f_senseval2, f_senseval3):
    altered_dataset.append(s)
    altered_dataset.append(s2)
    altered_dataset.append(s3)
#----------------
#storing all the texts for later training our model
sentences_ = []

for s in altered_dataset:
    text = " ".join([x.text for x in s])
    text = utils.simple_preprocess(text)
    #appending texts in every snetence
    sentences_.append(text)
    for word in text:
        syns = wn.synsets(word)
        for syn in syns:
            #---------
            definition = utils.simple_preprocess(syn.definition())
            if len(definition)>0:
                #appending all the defintions
                sentences_.append(definition)
            #------------------
            examples = " ".join(syn.examples())
            if len(examples)>0:
                examples = utils.simple_preprocess(examples)
                #appending all the examples
                sentences_.append(examples)
        

C:\Users\nabaw\AppData\Local\Temp\ipykernel_23188\1965273610.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f_semcor = np.random.choice( documents, size=50)
C:\Users\nabaw\AppData\Local\Temp\ipykernel_23188\1965273610.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f_senseval2 = np.random.choice( doc_senseval2, size=50)
C:\Users\nabaw\AppData\Local\Temp\ipykernel_23188\1965273610.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or

In [857]:
sentences_[0]

['houghton',
 'said',
 'had',
 'been',
 'set',
 'for',
 'canvass',
 'of',
 'all',
 'homes',
 'in',
 'the',
 'subdivision',
 'which',
 'is',
 'located',
 'northeast',
 'of',
 'dequindre',
 'and',
 'mile',
 'road',
 'east']

#### training the model

In [855]:
# load library gensim (contains word2vec implementation)
import gensim

# ignore some warnings (probably caused by gensim version)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import multiprocessing
cores = multiprocessing.cpu_count()


w2v_model_extnesion = gensim.models.Word2Vec(min_count=3,
                                   window=3,
                                   vector_size=300,  # this parameter is called 'vector_size' in more recent versions of gensim
                                   workers=cores,
                                   sg=1
                                   )

# defining the vocabulary based on our data
w2v_model_extnesion.build_vocab(sentences_, progress_per=10000)

# training
w2v_model_extnesion.train(sentences_, total_examples=w2v_model_extnesion.corpus_count, epochs=10, report_delay=1)
w2v_model_extnesion.init_sims(replace=True)

In [867]:
w2v_model_extnesion.wv["road","man"].shape

(2, 300)

In [864]:
"road" in w2v_model_extnesion.wv.index_to_key

True

as there are few changes in the word2vec model we trained and the pre-trained one, we need to rewrite the code a bit

In [881]:
from gensim import utils
from operator import itemgetter
from gensim import matutils

def get_acc(true_senses, result):
    all_lems = []
    for s in true_senses:
        for l in s.lemmas():
            all_lems.append(l.key())
    try:
        res = result[:-1][0]
        f = []
        for r in res:
            for k in r.split(","):
                if len(k)>0:
                    f.append(k)
        #res = set([res.lower() for res in res.split(",") if len(res)>0])
        res = set(f)
        all_lems = set(s.lower() for s in all_lems)
        if len(all_lems.intersection(res))>0:
            return 1
        else:
            return 0
    except:
        return None

def show_acc(df):
    res = df.apply(lambda x: get_acc(x.true_synset, x.result), axis=1).dropna()
    corr = res.sum()
    all = res.shape[0]

    return (corr/all)*100, corr, all

def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1.reshape(1,-1)), matutils.unitvec(vec2))
    return cosine_similarity

def lemma_mixer(lem, lex):
    lem = str(lem).lower()
    lex = str(lex).lower().split(";")
    for i in range(len(lex)):
        lex[i]= lem+"%"+lex[i]
    return ",".join(lex)

def gloss_vec(x, model):
    defs_ = []
    for syn in x:
        sc = utils.simple_preprocess(syn.definition())
        sc = [token for token in sc if token in model.wv.index_to_key and token not in stopwords.words('english')]
        try:
            sc = np.mean(model.wv[sc], axis=0)
        except:
            sc = np.zeros((1,300), dtype=float)
        defs_.append(sc)
    return defs_
    
def example_vec(x, model):
    defs_ = []
    for syn in x:
        sc = utils.simple_preprocess(".".join(syn.examples()))
        sc = [token for token in sc if token in model.wv.index_to_key and token not in stopwords.words('english')]
        try:
            sc = np.mean(model.wv[sc], axis=0)
        except:
            sc = np.zeros((1,300), dtype=float)
        defs_.append(sc)
    return defs_

def lemma_finder(wn_syn, lem_n):
    lem_n = lem_n[0].split("%")[0]+"%"
    lem_syns = []
    for lem in wn_syn:
        lems = []
        for lem in lem.lemmas():
            if str(lem.key()).startswith(lem_n):
                lems.append(lem.key())
        if len(lems)>0:
            lem_syns.append(lems)
    return lem_syns


from operator import itemgetter
def finder_l(x, w):
    xx = set(wx for wx in x.lower().split(",") if len(wx)>0)
    w = [y[0] for y in w]
    ww = set(x.lower() for x in w if len(x)>0)
    #looking for lexemes with lemma key.
    res = bool(xx & ww)
    if res:
        return True
    else:
        return False

def mapp_finder(df, w):
    return df[df["wordlemma"].apply(lambda x: finder_l(x, w))]

def lex_creator(key, lemkey):
    keyz = []
    lemkey = [y for y in lemkey.split(",") if len(y)>0]
    for x in lemkey:
        if len(x)>0:
            keyz.append(x.split("%")[0]+"-"+key)
    return keyz


def disambiguator(word, true_lemmas, lemmas, gloss, examples,model,  alpha, beta, lambda_):
    if word not in context:
        return None
    #if no disambiguated words exist
    if len(dis_vec)==0:
        contxt_vec = model.wv[context]
    #if we had disamibuated words earlier 
    else:                      
        alread_dis = np.concatenate(dis_vec, axis=0)
        contxt_vec = np.concatenate((alread_dis,model.wv[context]), axis=0)

    context_mean_vec = np.mean(contxt_vec, axis=0)
    #counter to catch the first word
    i = 0
    #highest cosine similarity
    cos_high = 0
    #best syns id
    syns_id = 0
    #syns of the chosen word
    syns_word_selected = 0
    #synset wordnet
    synset_wd = 0
    s = 0
    #lem_n = true_lemmas[0].split("%")[0]
    keyz = mapp_finder(mapp_df, lemmas)
    # keyz["lex_keyz"] = keyz.apply(lambda x: lex_creator(x.wkey, x.wordlemma), axis=1)
    #lemma key ---------------
    for lem_syn in lemmas:
        gloss_vec = gloss[s]
        examples_vec = examples[s]
        if (len(gloss_vec)>0 and alpha != 0):
            score_sw = similarity_cosine(gloss_vec.flatten(), context_mean_vec.flatten()) #syns_embedding
        else:
            score_sw = 0
        # keyz = mapp_finder(mapp_df, lem_syn)
        if (len(examples_vec)>0 and lambda_ != 0):
            score_example_s = similarity_cosine(examples_vec.flatten(), context_mean_vec.flatten()) 
        else:
            score_example_s = 0

        for syn in lem_syn:
            #----------------
            #exact map index
            
            map_idx = keyz[keyz.iloc[:,1].apply(lambda x: True if syn in x.lower().split(",") else False)]
            #finding synset embedding
            try:
                map_lemmakey = map_idx.iloc[:,0].tolist()[0]
            except:
                continue
            syns_embedding = synsets_emb[map_lemmakey]
            #--------------------
            if beta != 0:
                #finding lexeme------------
                lem_n = map_idx.iloc[:,1].tolist()[0].split("%")[0]
                lex_key = lem_n+"-"+map_lemmakey
                try:
                    lexeme_embedding = lexemes_emb[lex_key]
                    score_lsw = similarity_cosine(lexeme_embedding.flatten(), context_mean_vec.flatten())
                except:
                    #continue
                    score_lsw = 0#similarity_cosine(np.zeros((1,300), dtype=float).flatten(), context_mean_vec.flatten())
            else:
                score_lsw = 0
            #---------------------------------

            #final score
            score = alpha*score_sw + beta*score_lsw + lambda_*score_example_s
            if i == 0:
                cos_high = score
                syns_id = lem_syn#map_idx.iloc[:,1].tolist()[0]
                syns_word_selected = syns_embedding#syns_embedding
                #synset_wd = wn_syn
                i += 1
            if score>cos_high:
                cos_high = score
                syns_id = lem_syn#map_idx.iloc[:,1].tolist()[0]
                syns_word_selected = syns_embedding#syns_embedding    
                #synset_wd = wn_syn       
        s += 1     

    #if i was able to disambiguate the word add it so next time only the synset embedding is used
    if i>0:
        context.remove(word)
        dis_vec.append(np.array(syns_word_selected, dtype=float).reshape(1,-1))
    return [syns_id, cos_high]

def final_wsd_wv(docs, model=word2vec, alpha=0.5, beta=0.1, lambda_=0.9):
    full = []
    for doc in tqdm(docs):
    #one sentence
        one_sent = utils.simple_preprocess(" ".join([x.text for x in doc]))
    #a list to store word, lemmakey, number of senses, sense key, syns
        _sent = []
    #every word in the sentence
        for wrd in one_sent:
        #make lemma key
            lemk = [lemma_mixer(x.lemma, x.lexsn) for x in doc if (("".join(utils.simple_preprocess(x.text)) == wrd)and((x.lexsn != None) and (x.lemma != None)))]
        # if there is lexsn and lemma
        #if len(lemk)>0:
            try:
            #making the lemma key, true one given by sentence.
                lemk = ",".join(lemk).split(",")
                syns_t = [wn.lemma_from_key(l).synset() for l in lemk]
            except:
                continue
            #lemk = None
            #using wordnet, find all the synsets for the word
            syns = wn.synsets(wrd)

            _sent.append([wrd, lemk,syns_t, len(syns),  syns])
        if len(_sent)==0:
            continue
        global dis_vec
        dis_vec = []
        #sorting
        _sent = sorted(_sent, key=itemgetter(3))
        global context
        context = [token[0] for token in _sent if token[0] in model.wv.index_to_key and token[0] not in stopwords.words('english')]
        _sent = pd.DataFrame(_sent, columns=["word", "true_lemmas","true_synset", "frequency", "all_synsets"])
        _sent["gloss"] = _sent["all_synsets"].apply(lambda x: gloss_vec(x, model))
        _sent["examples"] = _sent["all_synsets"].apply(lambda x: example_vec(x, model))
        _sent["all_lemmas"]=_sent.apply(lambda x: lemma_finder(x.all_synsets, x.true_lemmas), axis=1)
        _sent["result"]=_sent.apply(lambda x: disambiguator(x.word, x.true_lemmas, x.all_lemmas, x.gloss, x.examples, model, alpha, beta, lambda_), axis=1)
        full.append(_sent)
    full = pd.concat(full)

    return full.dropna().reset_index(drop=True)

In [870]:
#extension
altered = final_wsd_wv(altered_dataset, model=w2v_model_extnesion, alpha=1, beta=1, lambda_=0)

100%|██████████| 150/150 [16:11<00:00,  6.48s/it]


In [871]:
show_acc(altered)

(28.621495327102803, 245.0, 856)

In [872]:
#extension
altered1 = final_wsd_wv(altered_dataset, model=w2v_model_extnesion, alpha=0.5, beta=0.1, lambda_=1.2)
show_acc(altered1)

100%|██████████| 150/150 [16:51<00:00,  6.74s/it]


(31.89252336448598, 273.0, 856)

In [873]:
#extension
altered3 = final_wsd_wv(altered_dataset, model=w2v_model_extnesion, alpha=0, beta=0.8, lambda_=0.5)
show_acc(altered3)

100%|██████████| 150/150 [16:10<00:00,  6.47s/it]


(35.981308411214954, 308.0, 856)

In [ ]:
#extension
altered4 = final_wsd_wv(altered_dataset, model=w2v_model_extnesion, alpha=0.8, beta=0, lambda_=0.9)


In [882]:
show_acc(altered4)

(30.724299065420563, 263.0, 856)

In [878]:
alt_cs = find_most_common_sense(altered_dataset)
get_acc(alt_cs)

(61.06194690265486, 552, 904)

In [879]:
lescs = lesk_mc(altered_dataset)
evaluate_accuracy(lescs)

100%|██████████| 150/150 [00:00<00:00, 164.38it/s]


(49.345417925478344, 490, 993)

### Sequential code of LESK++

In [185]:
from operator import itemgetter
def finder(x, w):
    xx = set(wx for wx in x.lower().split(",") if len(wx)>0)
    ww = set(x.lower() for x in w if len(x)>0)
    #looking for lexemes with lemma key.
    res = bool(xx & ww)
    if res:
        return True
    else:
        return False

def mapp_finder(df, w):
    return df[df.iloc[:,-1].apply(lambda x: finder(x, w))]

def lemma_mixer(lem, lex):
    lem = str(lem).lower()
    lex = str(lex).lower().split(";")
    for i in range(len(lex)):
        lex[i]= lem+"%"+lex[i]
    return ",".join(lex)


from gensim import matutils
def similarity_cosine(vec1, vec2):
    cosine_similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
    return cosine_similarity

def wsd_pb(docs):
    full_sent = []
    for doc in tqdm(docs):
        #one sentence
        one_sent = utils.simple_preprocess(" ".join([x.text for x in doc]))
        #a list to store word, lemmakey, number of senses, sense key, syns
        _sent = []
        #every word in the sentence
        for wrd in one_sent:
            #make lemma key
            lemk = [lemma_mixer(x.lemma, x.lexsn) for x in doc if (("".join(utils.simple_preprocess(x.text)) == wrd)and((x.lexsn != None) and (x.lemma != None)))]
            # if there is lexsn and lemma
            if len(lemk)>0:
                #making the lemma key, true one given by sentence.
                lemk = ",".join(lemk)
            else:
                continue
                #lemk = None
            #using wordnet, find all the synsets for the word
            syns = wn.synsets(wrd)
            _sent.append([wrd,lemk, len(syns), syns])
        #sorting
        _sent = sorted(_sent, key=itemgetter(2))

        #vector for disambiguated words
        dis_vec = []

        context = [token[0] for token in _sent if token[0] in word2vec.index_to_key and token[0] not in stopwords.words('english')]# and token[0] not in stopwords.words('english')
        # try:
        for ambigus_w in _sent:
            #if we don't have the word in the context skip it
            if ambigus_w[0] not in context:
                continue
            #counter to catch the first word
            i = 0
            #highest cosine similarity
            cos_high = 0
            #best syns id
            syns_id = 0
            #syns of the chosen word
            syns_word_selected = 0
            #synset wordnet
            synset_wd = 0
            lem_n = ambigus_w[1].split("%")[0]
            for wn_syn in ambigus_w[3]:
                #gloss-------------------------------------------
                gloss = [w for w in wn_syn.definition().split() if w in word2vec.index_to_key and w not in stopwords.words('english')]
                if len(gloss)>0:
                    gloss_vec = np.mean(word2vec[gloss], axis=0)
                else:
                    gloss_vec = np.zeros((1,300), dtype=float)
                #--------------------------------------------------

                #getting all the lemas-----------------
                lems = []
                for lem in wn_syn.lemmas():
                    if str(lem.key()).startswith(lem_n):
                        lems.append(lem.key())
                #----------------
                #if there is no lemma go to the next synset---
                if len(lems)==0:
                    continue
                #-------------------
                #only choosing the lemmaskey existing this synset
                keyz = mapp_finder(mapp_df, lems)
                #-------------------

                #if no disambiguated words exist
                if len(dis_vec)==0:
                    contxt_vec = word2vec[context]
                #if we had disamibuated words earlier 
                else:                      
                    alread_dis = np.concatenate(dis_vec, axis=0)
                    contxt_vec = np.concatenate((alread_dis,word2vec[context]), axis=0)

                context_mean_vec = np.mean(contxt_vec, axis=0)
                score_sw = similarity_cosine(gloss_vec.flatten(), context_mean_vec.flatten()) #syns_embedding

                #lemma key ---------------
                for syn in lems:
                    #----------------
                    #exact map index
                    map_idx = keyz[keyz.iloc[:,1].apply(lambda x: True if syn in x.lower().split(",") else False)]
                    #finding synset embedding
                    try:
                        map_lemmakey = map_idx.iloc[:,0].tolist()[0]
                    except:
                        continue
                    syns_embedding = synsets_emb[map_lemmakey]
                    #--------------------

                    #finding lexeme------------
                    lex_key = lem_n+"-"+map_lemmakey
                    try:
                        lexeme_embedding = lexemes_emb[lex_key]
                        score_lsw = similarity_cosine(lexeme_embedding.flatten(), context_mean_vec.flatten())
                    except:
                        score_lsw = similarity_cosine(np.zeros((1,300), dtype=float).flatten(), context_mean_vec.flatten())
                    #---------------------------------

                    #final score
                    score = score_sw+score_lsw
                    if i == 0:
                        cos_high = score
                        syns_id = map_idx.iloc[:,1].tolist()[0]
                        syns_word_selected = syns_embedding
                        #synset_wd = wn_syn
                        i += 1
                    if score>cos_high:
                        cos_high = score
                        syns_id = map_idx.iloc[:,1].tolist()[0]
                        syns_word_selected = syns_embedding    
                        #synset_wd = wn_syn            

            #if i was able to disambiguate the word add it so next time only the synset embedding is used
            if i>0:
                context.remove(ambigus_w[0])
                dis_vec.append(np.array(syns_word_selected, dtype=float).reshape(1,-1))
            try:
                full_sent.append([ambigus_w[0], ambigus_w[1],syns_id, cos_high])
            except:
                pass
    return full_sent

In [186]:
sa = wsd_pb(doc_senseval2)

100%|██████████| 238/238 [2:41:15<00:00, 40.66s/it]  


In [242]:
sa[0]

['unintelligible',
 'unintelligible%5:00:00:incomprehensible:00',
 'unintelligible%3:00:00::,',
 0.77900994]

In [192]:
get_acc_wsd(sa)

(50.23746701846966, 1895, 952)

In [193]:
sb = wsd_pb(doc_senseval3)

100%|██████████| 300/300 [2:18:03<00:00, 27.61s/it]  


In [194]:
get_acc_wsd(sb)

(33.37378640776699, 1648, 550)

In [ ]:
sc = wsd_pb(sample_semcor)

In [ ]:
get_acc_wsd(sc)

In [191]:
def get_acc_wsd(results):
    all = 0
    match = 0
    for w in results:
        all += 1
        for syns in wn.synsets(w[0]):
            lems = []
            for l in syns.lemmas():
                lems.append(l.key())
            try:
                res_orig = set(w[1].lower().split(",")).intersection(set(lems))
                res_pred = set(w[2].lower().split(",")).intersection(set(lems))
                if bool(res_orig & res_pred):
                    match += 1
            except:
                all -= 1
                continue

    return (match/all)*100, all, match
        